In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from itertools import product
from IPython.display import display
from glob import glob

In [ ]:
# res = pd.read_csv('./logvsnolog/experiment_rosen_loglaurens_iters.csv')
# res = pd.concat([
#     pd.read_csv('./rosenvar/experiment_rosen_nolog_iters.csv'),
#     pd.read_csv('./rosenvar/experiment_rosen_log_iters.csv')])
res = pd.concat(pd.read_csv(f) for f in glob("../results/results-rosen238-tiny/*_iters.csv"))
res

In [ ]:
def gen_samplemat(col):
    return np.vstack(col.apply(lambda x: np.asarray(x.strip("][").split(', '), dtype=float)))

In [ ]:
allsamples = gen_samplemat(res['iter_x'])
dmin = np.min(allsamples, axis=0)
dmax = np.max(allsamples, axis=0)

display(allsamples)
display(dmin)
display(dmax)

def gensamplemat_norm(col):
    return (gen_samplemat(col) - dmin) / (dmax - dmin)

In [ ]:
expmats = res.groupby('exp_id')['iter_x'].agg(lambda x: (gensamplemat_norm(x), )).apply(lambda x: x[0])
expmats.name = 'samplemat'
expmats = pd.merge(expmats.reset_index(), res[res['iter_idx']==0][['exp_id', 'approach', 'problem']], how='left')
expmats

In [ ]:
# iis = range(0, 1000, 100)
iis = (0, 7, 14, 21)
#iis = (0, 100, 200, 300, 450, 600, 650, 750, 850)
# iis = (200, 201, 202, 203)
d = 5
nplotsperi = 3

fig, axs = plt.subplots(len(iis), nplotsperi, figsize=(18 / 3 * nplotsperi, 16/4 * len(iis)))

for pltidx, i in enumerate(iis):
    # i= 300 (randomsearch), 0 (idone/advanced), 100 (idone/basic)
    # i = 200
    r = expmats.iloc[i]
    islog = 'log=True' in r['problem']
    print(f"{r['problem']}: islog? {islog}")

    smd = r['samplemat'][:, d]
    ax1 = axs[pltidx, 0]
    ax1.plot(smd)
    ax1.title.set_text(f"i: {i}, approach: {r['approach']}, log?:{islog}, d: {d}")
    # plt.show()

    smdd = np.convolve(smd, [-1, 1])
    # plt.plot(smdd)
    ax2 = axs[pltidx, 1]
    ax2.hist(smdd, bins=np.linspace(-1.0, 1.0, 20))
    ax2.title.set_text(f"i: {i}, approach: {r['approach']}, log?:{islog}, d: {d}")
    # plt.show()

    ax3 = axs[pltidx, 2]
    smdfft = np.abs(np.fft.fft(smd - np.mean(smd)))
    ax3.plot(smdfft)
    ax3.title.set_text(f"i: {i}, approach: {r['approach']}, log?:{islog}, d: {d} / fft")

    # plt.plot(np.abs(np.fft.fft(r['samplemat'][:, d] - np.mean(r['samplemat'][:, d]))))
    # plt.title(f"i: {i}, approach: {r['approach']}, d: {d} / fft")
    # plt.show()
    # plt.show()

fig.show()

In [ ]:
# iis = (0, 100, 200, 300, 400)
# iis = (range(0, 5+1), range(100, 105+1), range(200, 205+1), range(300, 305+1), range(450, 455+1))
iis = [range(i, i+7) for i in range(0, 28, 7)]
nplotsperi = 3
d = 200

fig, axs = plt.subplots(len(iis), nplotsperi, figsize=(18, 16))

smds = []
smdffts = []

for pltidx, ix in enumerate(iis):
    # i= 300 (randomsearch), 0 (idone/advanced), 100 (idone/basic)
    # i = 200
    i = ix[0]
    r = expmats.iloc[i]
    
    smd = np.stack(expmats.iloc[ix]['samplemat'])[:, 0:24, d]
    ax1 = axs[pltidx, 0]
    ax1.plot(smd.T)
    ax1.title.set_text(f"i: {i}, approach: {r['approach']}, d: {d}")

    smds.append(smd)
    # plt.show()

    smdd = np.apply_along_axis(lambda x: np.convolve(x, [-1, 1]), 1, smd)
    # plt.plot(smdd)
    ax2 = axs[pltidx, 1]
    ax2.hist(smdd.T, bins=np.linspace(-1.0, 1.0, 20))
    ax2.title.set_text(f"i: {i}, approach: {r['approach']}, d: {d}")
    # plt.show()
    
    ax3 = axs[pltidx, 2]
    smdfft = np.abs(np.fft.fft(smd, axis=1))[:, 1:]
    # smdfft[:, 0] = 0.0
    # ax3.plot(np.mean(smdfft.T,axis=1))
    ax3.plot(smdfft.T)
    ax3.title.set_text(f"i: {i}, approach: {r['approach']}, d: {d} / fft")

    # smdffts.append(smdfft)
    smdffts.append(np.mean(smdfft.T,axis=1))

    # plt.plot(np.abs(np.fft.fft(r['samplemat'][:, d] - np.mean(r['samplemat'][:, d]))))
    # plt.title(f"i: {i}, approach: {r['approach']}, d: {d} / fft")
    # plt.show()
    # plt.show()

fig.show()

In [ ]:
# iis = (0, 100, 200, 300, 400)
iis = (range(0, 5+1), range(100, 105+1), range(200, 205+1), range(300, 305+1), range(400, 405+1))
nplotsperi = 3

fig, axs = plt.subplots(len(iis), nplotsperi, figsize=(18, 16))

smds = []
smdffts = []

for pltidx, ix in enumerate(iis):
    # i= 300 (randomsearch), 0 (idone/advanced), 100 (idone/basic)
    # i = 200
    d = 5
    i = ix[0]
    r = expmats.iloc[i]
    
    smd = np.stack(expmats.iloc[ix]['samplemat'])[:, :, d]
    ax1 = axs[pltidx, 0]
    ax1.plot(smd.T)
    ax1.title.set_text(f"i: {i}, approach: {r['approach']}, d: {d}")

    smds.append(smd)
    # plt.show()

    smdd = np.apply_along_axis(lambda x: np.convolve(x, [-1, 1]), 1, smd)
    # plt.plot(smdd)
    ax2 = axs[pltidx, 1]
    ax2.hist(smdd.T, bins=np.linspace(-1.0, 1.0, 20))
    ax2.title.set_text(f"i: {i}, approach: {r['approach']}, d: {d}")
    # plt.show()
    
    ax3 = axs[pltidx, 2]
    smdfft = np.abs(np.fft.fft(smd, axis=1))[:, 1:]
    # smdfft[:, 0] = 0.0
    ax3.plot(np.mean(smdfft.T,axis=1))
    # ax3.plot(smdfft.T)
    ax3.title.set_text(f"i: {i}, approach: {r['approach']}, d: {d} / fft")

    # smdffts.append(smdfft)
    smdffts.append(np.mean(smdfft.T,axis=1))

    # plt.plot(np.abs(np.fft.fft(r['samplemat'][:, d] - np.mean(r['samplemat'][:, d]))))
    # plt.title(f"i: {i}, approach: {r['approach']}, d: {d} / fft")
    # plt.show()
    # plt.show()

fig.show()

In [ ]:
np.asarray([ np.mean(
        np.minimum(smdffts[i], smdffts[j]) / 
        np.maximum(smdffts[i], smdffts[j])) 
    for i in range(0, len(smdffts))
    for j in range(0, len(smdffts))]).reshape((len(smdffts), len(smdffts)))

In [ ]:
smd = expmats.iloc[106]['samplemat'][:, d]
smdfft = np.abs(np.fft.fft(smd, axis=0))[1:]
np.asarray([ np.mean(
        np.minimum(smdffts[i], smdfft) / 
        np.maximum(smdffts[i], smdfft)) 
    for i in range(0, len(smdffts))])

In [ ]:
def fftvar(samplemat):
    smfft = np.fft.fft(samplemat, axis=1)
    smfftl = np.abs(smfft)
    return np.mean(smfftl)

def meanstd(samplemat):
    return np.mean(np.std(samplemat, axis=1))

In [ ]:
expmats['variances'] = expmats['samplemat'].apply(fftvar)
expmats.groupby('approach')['variances'].mean()

In [ ]:
expmats['variances'] = expmats['samplemat'].apply(meanstd)
expmats.groupby('approach')['variances'].mean()

In [ ]:
min_v = 0
max_v = 10
n_evals = 200
n_samples = 1000000

rng = np.random.default_rng(0)
unif_samples = rng.integers(min_v, max_v, endpoint=True, size=(n_evals, n_samples))
unif_samples_norm = (unif_samples - min_v) / (max_v - min_v)
unif_samples_fft = np.abs(np.fft.fft(unif_samples_norm - np.mean(unif_samples_norm), axis=0))[1:]
unif_samples_fft_mean = np.mean(unif_samples_fft, axis=1)

plt.plot(unif_samples_fft_mean)
plt.show()

In [ ]:
min_v = 0
max_v = 10
n_evals = 400
n_samples = 1000000

rng = np.random.default_rng(0)
unif_samples = rng.integers(min_v, max_v, endpoint=True, size=(n_evals, n_samples))
unif_samples_norm = (unif_samples - min_v) / (max_v - min_v)
unif_samples_fft = np.abs(np.fft.fft(unif_samples_norm - np.mean(unif_samples_norm), axis=0))[1:]
unif_samples_fft_mean = np.mean(unif_samples_fft, axis=1)

plt.plot(unif_samples_fft_mean)
plt.show()

In [ ]:
min_v = 0
max_v = 10
n_evals = 400
n_samples = 10000
p = 0.01

rng = np.random.default_rng(0)

unif_samples = min_v + rng.binomial(max_v - min_v, p, size=(n_evals, n_samples))
unif_samples_norm = (unif_samples - min_v) / (max_v - min_v)
plt.plot(unif_samples_norm[:, 1])
plt.show()
unif_samples_fft = np.abs(np.fft.fft(unif_samples_norm - np.mean(unif_samples_norm), axis=0))[1:]
unif_samples_fft_mean = np.mean(unif_samples_fft, axis=1)

plt.plot(unif_samples_fft_mean)
plt.show()

In [ ]:
min_v + rng.binomial(max_v - min_v, 0.1)

In [ ]:
def fnl(Ex, N, k):
    return np.abs(Ex) * np.abs(sum(np.exp(-2j*np.pi*k*n/N) for n in range(0, N-1)))

fnl(1/10*p, 400, 200)

In [ ]:
N = 400
k = 200
np.abs(sum(np.exp(-2j*np.pi*k*n/N) for n in range(0, N-1)))
